In [176]:
import pandas as pd
import numpy as np
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import ast

In [177]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [178]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r"\d+", '', text)
    text = ' '.join([word for word in text.split() if ((word not in stop_words) and (len(word) >= 3))])
    return text

In [179]:
df = pd.read_csv('C:\\Users\\joaov\\Documents\\Whisky Recomendation System\\Whisky Notes Extraction\\Step 2\\Whisky Notes - Step 2.csv', sep=';')
df

,review_date,review_title,review_link,review_full_text,review_image,review_score
0,2025-10-03,"A Secret Highland Distillery 14 yo 2010 (54,1%...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: a lot of sherry influence, but never in-...",https://www.whiskynotes.be/wp-content/uploads/...,84
1,2025-10-03,"Inchgower 13 yo 2012 (56,4%, Whiskydudes 2015,...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: lots of butter biscuits at first, even a...",https://www.whiskynotes.be/wp-content/uploads/...,86
2,2025-10-03,"Secret Campbeltown Blended Malt 8 yo 2016 (58,...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: nice gristy notes, some balanced maritim...",https://www.whiskynotes.be/wp-content/uploads/...,88
3,2025-10-03,"Caol Ila 9 yo 2014 (57,6%, Whiskydudes 2024, P...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: rich and full, albeit with a surprisingl...",https://www.whiskynotes.be/wp-content/uploads/...,88
4,2025-10-03,"Caol Ila 10 yo 2015 (55,6%, Whiskydudes 2025, ...",https://www.whiskynotes.be/2025/caol-ila/whisk...,Nose: sour wine and sour berries come out firs...,https://www.whiskynotes.be/wp-content/uploads/...,84
...,...,...,...,...,...,...
5828,2008-12-11,"Clynelish 12y (57%, G&M 1980’s – Ainslie & Hei...",https://www.whiskynotes.be/2008/clynelish/clyn...,"Nose: dried fruits (plums, raisins). Pine fore...",https://www.whiskynotes.be/upload/Clynelish12y...,93
5829,2008-12-10,"Banff 25y 1980 (56,5%, DT Rarest of the Rare 2...",https://www.whiskynotes.be/2008/banff/banff-25...,"Apricot and frangipane on the nose. Cake, vani...",https://www.whiskynotes.be/upload/Banff25y1980...,84
5830,2008-12-09,"Teaninich 23y 1972 (64,95%, Rare Malts 1996)",https://www.whiskynotes.be/2008/teaninich/tean...,"Interesting nose: wood, crème de banane, beesw...",https://www.whiskynotes.be/upload/Teaninich23y...,90
5831,2008-12-08,"Craigduff 32y 1973 (49,4%, SV 2005, cask #2513...",https://www.whiskynotes.be/2008/glen-keith/cra...,"Beautiful nose, with notes of marzipan, ginger...",https://www.whiskynotes.be/upload/Craigduff32y...,86


In [180]:
df = df[['review_title', 'review_full_text']]

In [181]:
#df['review_title'] = df['review_title'].str.lower().str.replace('-', ' ')
df['review_full_text'] = df['review_full_text'].str.lower().str.replace('-', ' ')

C:\Users\joaov\AppData\Local\Temp\ipykernel_9660\3665824303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_full_text'] = df['review_full_text'].str.lower().str.replace('-', ' ')


## For future usage

In [182]:
# df['age'] = df['review_title'].str.extract(r'(\d+)\s*yo', flags=re.I).astype('Int64')
# df['abv'] = df['review_title'].str.extract(r'(\d+[,.]?\d*)\s*%', flags=re.I)
# df['abv'] = df['abv'].replace(',', '.', regex=True).astype(float)


In [183]:
# # For future usage

# df['fill'] = np.where((df['review_title'].str.contains('first fill|fresh fill', case=False, na=False)), 'First Fill',
#              np.where((df['review_title'].str.contains('second fill|refill', case=False, na=False)), 'Second Fill',
#              np.where((df['review_title'].str.contains('third fill', case=False, na=False)), 'Third Fill',
#              None)))
                    
# df['previous_spirit'] = np.where((df['review_title'].str.contains('sherry|jerez|xerez|pedro ximenez|px|oloroso|fino|amontillado|palo cortado', case=False, na=False)), 'Sherry',
#                         np.where((df['review_title'].str.contains('bourbon', case=False, na=False)), 'Bourbon',
#                         np.where((df['review_title'].str.contains('madeira', case=False, na=False)), 'Madeira',
#                         np.where((df['review_title'].str.contains('port', case=False, na=False)), 'Port',
#                         np.where((df['review_title'].str.contains('wine', case=False, na=False)), 'Wine',
#                         np.where((df['review_title'].str.contains('cognac|armagnac', case=False, na=False)), 'Cognac/Armagnac',
#                         np.where((df['review_title'].str.contains('rum', case=False, na=False)), 'Rum',
#                         np.where((df['review_title'].str.contains('virgin', case=False, na=False)), 'Virgin Oak',
#                         np.where((df['review_title'].str.contains('amarone', case=False, na=False)), 'Amarone',
#                         None)))))))))

In [184]:
# dim_distilleries = pd.read_csv('dim_distilleries.csv', sep=';')
# dim_distilleries

In [185]:
# dim_distilleries = dim_distilleries[dim_distilleries['Is Closed'] == False].reset_index(drop=True)
# dim_distilleries.columns = dim_distilleries.columns.str.strip().str.replace(' ', '_').str.lower()
# dim_distilleries = dim_distilleries[['distillery', 'region', 'type_of_whisky','latitude','longitude']]
# dim_distilleries['distillery'] = dim_distilleries['distillery'].str.lower()
# dim_distilleries

In [186]:
# distilleries_list = dim_distilleries['distillery'].tolist()
# df['distillery'] = np.nan

# for distillery in distilleries_list:
#     df.loc[df["review_title"].str.contains(fr'(^|[\s,;.!?]){distillery}($|[\s,;.!?])', case=False, na=False), 'distillery'] = distillery

In [187]:
# df = df.merge(dim_distilleries, on='distillery', how='left')

In [188]:
# df['type_of_spirit'] = np.where(df['review_title'].str.contains('cognac', na=False), 'Cognac',
#                        np.where(df['review_title'].str.contains('armagnac', na=False), 'Armagnac',
#                        np.where(df['review_title'].str.contains('rum', na=False), 'Rum',
#                        np.where(df['review_title'].str.contains('eau-de-vie', na=False), 'Eau-de-Vie',
#                        np.where(df['review_title'].str.contains('new make', na=False), 'New Make',
#                        'Whisky')))))

In [189]:
# df['cask_size'] = np.where(df['review_title'].str.contains('hogshead', na=False), 'Hogshead',
#                   np.where(df['review_title'].str.contains('barrel', na=False), 'Barrel',
#                   np.where(df['review_title'].str.contains('butt', na=False), 'Butt',
#                   np.where(df['review_title'].str.contains('puncheon', na=False), 'Puncheon',
#                   np.where(df['review_title'].str.contains('quarter cask', na=False), 'Quarter Cask',
#                   np.where(df['review_title'].str.contains('octave', na=False), 'Octave',
#                   np.where(df['review_title'].str.contains('cask', na=False), 'Cask',
#                   None)))))))

In [190]:
# df['cask_fill'] = np.where(df['review_title'].str.replace('-', ' ').str.contains('first fill', na=False), 'First Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('fresh fill', na=False), 'First Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('second fill', na=False), 'Second Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('refill', na=False), 'Second Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('third fill', na=False), 'Third Fill',
#                   None)))))

In [191]:
# df['previous_spirit'] = np.where(df['review_title'].str.replace('-', ' ').str.contains('bourbon', na=False), 'Bourbon',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('sherry', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('oloroso', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('px', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('pedro ximénez', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('palo cortado', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('amontillado', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('fino', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('wine', na=False), 'Wine',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('amarone', na=False), 'Wine',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('rivesaltes', na=False), 'Wine',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('rum', na=False) & (df['type_of_spirit'] != 'Rum'), 'Rum',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('cognac', na=False) & (df['type_of_spirit'] != 'Cognac'), 'Cognac',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('armagnac', na=False) & (df['type_of_spirit'] != 'Armagnac'), 'Armagnac',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('port', na=False), 'Port',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('tequila', na=False), 'Tequila',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('madeira', na=False), 'Madeira',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('vermouth', na=False), 'Vermouth',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('virgin oak', na=False), 'Virgin Oak',
#                         None)))))))))))))))))))

## -----

In [192]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

C:\Users\joaov\AppData\Local\Temp\ipykernel_9660\3528310598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [193]:
df.loc[:, 'review_full_text'] = df['review_full_text'].str.replace('nose:|mouth:|finish:', '', regex=True)

In [194]:
parser_porter_stemmer = PorterStemmer()

def stemmer(text):

    y = []

    text_words = text.split()

    for word in text_words:
        y.append(parser_porter_stemmer.stem(word))
    
    stemmed_text = ' '.join(y)

    return stemmed_text

# parser_sb_stemmer = SnowballStemmer(language='english')

# def stemmer(text):

#     text_words = text

#     groups = {}

#     for word in text_words:

#         root = parser_sb_stemmer.stem(word)

#         if root not in groups:
#             groups[root] = []

#         groups[root].append(word)
    
#     clean_words = [min(group, key=len) for group in groups.values()]

#     clean_words = sorted(set(clean_words))

#     stemmed_text = ' '.join(clean_words)

#     return stemmed_text

In [195]:
df.loc[:, 'review_full_text_processed'] = [preprocess_text(text) for text in df['review_full_text']]

C:\Users\joaov\AppData\Local\Temp\ipykernel_9660\2149076747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'review_full_text_processed'] = [preprocess_text(text) for text in df['review_full_text']]


In [196]:
df.loc[:, 'review_full_text_processed'] = df['review_full_text_processed'].apply(stemmer)

In [197]:
normalize_map = {
    "add": "add",
    "almond": "almond",
    "alcohol": "alcohol",
    "appear": "appear",
    "appl": "apple",
    "apricot": "apricot",
    "ash": "ash",
    "banana": "banana",
    "berri": "berry",
    "best": "best",
    "better": "better",
    "blackberri": "blackberry",
    "bring": "bring",
    "butter": "butter",
    "butteri": "butter",
    "candi": "candy",
    "caramel": "caramel",
    "cask": "cask",
    'chalki': 'chalk',
    "char": "char",
    "chocol": "chocolate",
    "cigar": "cigar",
    "cinnamon": "cinnamon",
    "citru": "citrus",
    "citrusi": "citrus",
    'cherri': 'cherry',
    "clove": "clove",
    "coffe": "coffee",
    "coffee": "coffee",
    "come": "come",
    "cream": "cream",
    "creami": "cream",
    "dark": "dark",
    "darker": "dark",
    "distilleri": "distillery",
    "dri": "dry",
    "drier": "dry",
    "dry": "dry",
    'dusti': 'dust',
    'earthi': 'earthy',
    "eucalyptu": "eucalyptus",
    "fade": "fade",
    "find": "find",
    "finish": "finish",
    "flower": "floral",
    "floral": "floral",
    "fragrant": "aroma",
    "fruit": "fruit",
    "fruiti": "fruit",
    "full": "full",
    "get": "get",
    "ginger": "ginger",
    "gooseberri": "gooseberry",
    "good": "good",
    "grain": "grain",
    "graini": "grain",
    "grape": "grape",
    "grapefruit": "grapefruit",
    "grass": "grass",
    "grassi": "grass",
    "great": "great",
    "grow": "grow",
    "hazelnut": "hazelnut",
    "heather": "heather",
    "herb": "herb",
    "herbal": "herb",
    "honey": "honey",
    "intens": "intense",
    "iodin": "iodine",
    'leafi': 'leaf',
    'leatheri': 'leather',
    "length": "length",
    'liquoric': 'liqueur',
    "light": "light",
    "lightli": "light",
    "linger": "linger",
    "liqueur": "liqueur",
    "long": "long",
    "juici": "juicy",
    "juic": "juicy",
    "make": "make",
    "malt": "malt",
    "malti": "malt",
    "mango": "mango",
    "marzipan": "marzipan",
    "milk": "milk",
    'miner': 'mineral',
    "mint": "mint",
    "minti": "mint",
    "move": "move",
    "nut": "nut",
    "nutmeg": "nutmeg",
    "nutti": "nut",
    "oak": "oak",
    "oaki": "oak",
    "oil": "oily",
    "oily": "oily",
    "oili": "oily",
    "orange": "orange",
    "orang": "orange",
    'passion': 'passion fruit',
    "pastri": "pastry",
    "peach": "peach",
    "pear": "pear",
    "peat": "peat",
    "peati": "peat",
    "pepper": "pepper",
    "pepperi": "pepper",
    "peppercorn": "peppercorn",
    "pineappl": "pineapple",
    "pink": "pink",
    "plum": "plum",
    "raspberri": "raspberry",
    "red": "red",
    "resin": "resin",
    "rum": "rum",
    "round": "round",
    'salti': 'salt',
    "say": "say",
    "seem": "seem",
    "shake": "smoke",
    "sherri": "sherry",
    "sherry": "sherry",
    "show": "show",
    "smell": "aroma",
    "smoke": "smoke",
    "smoki": "smoke",
    "smooth": "smooth",
    "soft": "soft",
    "spice": "spice",
    "spici": "spice",
    "spirit": "spirit",
    "strength": "strength",
    "strong": "strong",
    "sugar": "sugar",
    "sweet": "sweet",
    "sweeter": "sweet",
    "syrup": "syrup",
    "take": "take",
    "tea": "tea",
    "toast": "toast",
    "toffe": "toffee",
    "tobacco": "tobacco",
    "tri": "try",
    "try": "try",
    "vanilla": "vanilla",
    "wax": "wax",
    "waxi": "wax",
    "white": "white",
    "whiski": "whisky",
    "wine": "wine",
    "wood": "wood",
    "woodi": "wood",
    "yellow": "yellow",
    "zest": "zest",
    "zesti": "zest"
}

def normalize_words(phrase):
    phrase = phrase.split()
    normalized_phrase = [normalize_map.get(word, word) for word in phrase]
    return " ".join(normalized_phrase)

df["review_full_text_processed"] = df["review_full_text_processed"].apply(normalize_words)

C:\Users\joaov\AppData\Local\Temp\ipykernel_9660\3794071298.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review_full_text_processed"] = df["review_full_text_processed"].apply(normalize_words)


In [198]:
counter = CountVectorizer(max_df=0.8, min_df=0.05, strip_accents='unicode', binary=True)
counter

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,'unicode'
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [ ]:
counter.fit_transform(df['review_full_text_processed']).toarray()

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5833, 285))

In [200]:
reference_words = counter.get_feature_names_out()
reference_words

array(['add', 'age', 'alcohol', 'almond', 'almost', 'along', 'alongsid',
       'also', 'although', 'alway', 'anise', 'appear', 'apple', 'apricot',
       'aroma', 'aromat', 'around', 'ash', 'avail', 'back', 'background',
       'bad', 'bake', 'balanc', 'banana', 'barley', 'beauti', 'becom',
       'berry', 'best', 'better', 'big', 'bit', 'bitter', 'black',
       'blackberry', 'bottl', 'bourbon', 'bread', 'bright', 'brine',
       'bring', 'brown', 'burnt', 'butter', 'cake', 'candy', 'caramel',
       'cask', 'certainli', 'chalk', 'char', 'charact', 'cherry',
       'chocolate', 'cigar', 'cinnamon', 'citrus', 'classic', 'clean',
       'close', 'clove', 'coastal', 'cocoa', 'coconut', 'coffee',
       'combin', 'come', 'complex', 'could', 'cours', 'cream', 'custard',
       'dark', 'definit', 'differ', 'distillery', 'dram', 'drop', 'dry',
       'dust', 'earthy', 'easi', 'edg', 'eleg', 'element', 'end',
       'eucalyptus', 'even', 'excel', 'exchang', 'expect', 'express',
       'fade'

In [201]:
tasting_notes_words_filtered = [
    'almond',
    'anise',
    'apple',
    'apricot',
    'ash',
    'banana',
    'barley',
    'berry',
    'bitter',
    'blackberry',
    'bread',
    'brine',
    'burnt',
    'butter',
    'cake',
    'candy',
    'caramel',
    'chalk',
    'char',
    'cherry',
    'chocolate',
    'cigar',
    'cinnamon',
    'citrus',
    'clove',
    'cocoa',
    'coconut',
    'coffee',
    'cream',
    'custard',
    'dust',
    'earthy',
    'eucalyptus',
    'fig',
    'floral',
    'fresh',
    'fruit',
    'ginger',
    'gooseberry',
    'grain',
    'grape',
    'grapefruit',
    'grass',
    'green',
    'hay',
    'hazelnut',
    'heather',
    'herb',
    'honey',
    'iodine',
    'jam',
    'juicy',
    'leather',
    'lemon',
    'lime',
    'liqueur',
    'malt',
    'mango',
    'marmalad',
    'marzipan',
    'medicin',
    'melon',
    'menthol',
    'milk',
    'mineral',
    'mint',
    'nut',
    'nutmeg',
    'oak',
    'oily',
    'orange',
    'passion fruit',
    'pastry',
    'peach',
    'pear',
    'peat',
    'peel',
    'pepper',
    'peppercorn',
    'pine',
    'pineapple',
    'plum',
    'raisin',
    'raspberry',
    'resin',
    'ripe',
    'roast',
    'rum',
    'salt',
    'sherry',
    'smoke',
    'sour',
    'spice',
    'stew',
    'sugar',
    'sweet',
    'syrup',
    'tangerin',
    'tea',
    'toast',
    'tobacco',
    'toffee',
    'tropic',
    'vanilla',
    'varnish',
    'walnut',
    'warm',
    'wax',
    'wine',
    'wood',
    'zest'
]

In [202]:
counter = CountVectorizer(vocabulary=tasting_notes_words_filtered, max_df=0.8, min_df=0.05, strip_accents='unicode', binary=True)
counter

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,'unicode'
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [203]:
review_text_vec = counter.fit_transform(df['review_full_text_processed']).toarray()
review_text_vec

array([[0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5833, 111))

In [204]:
df['review_text_vector'] = review_text_vec.tolist()

C:\Users\joaov\AppData\Local\Temp\ipykernel_9660\2636747614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_text_vector'] = review_text_vec.tolist()


In [205]:
df_tasting_notes = []

for i in df.index:

    review_title = df.loc[i, 'review_title']

    for j in range(len(tasting_notes_words_filtered)):

        tasting_note = tasting_notes_words_filtered[j]

        df_tasting_note = {
            'review_title': review_title,
            'tasting_note': tasting_note,
            'has_tasting_note': df.loc[i, 'review_text_vector'][j]
        }

        df_tasting_notes.append(df_tasting_note)

df_tasting_notes = pd.DataFrame(df_tasting_notes)

In [206]:
tasting_notes_groups = {

    "fruity": [
        "apple", "apricot", "banana", "berry", "blackberry", "fig", "grape",
        "grapefruit", "lemon", "lime", "mango", "melon", "orange", "peach",
        "pear", "pineapple", "plum", "raisin", "jam", "marmalad",
        "gooseberry", "grass", "green", "passion fruit", "tropic",
        "citrus", "tangerin", "zest", "juicy", 'gooseberry', 'fruit',
        "ripe", "stew", "peel", 'sour'
    ],

    "floral": [
        "floral", "heather", "herb", "leafy", "fresh"
    ],

    "spicy": [
        "anise", "cinnamon", "clove", "ginger", "nutmeg", "pepper",
        "peppercorn", "warm"
    ],

    "woody": [
        "oak", "wood", "pine", "resin", "varnish", "bitter"
    ],

    "cereal": [
        "barley", "bread", "cake", "grain", "malt", "yeast", "pastry", 'dust'
    ],

    "peaty": [
        "peat", "smoke", "ash", "burnt", "char", "medicin", "iodine"
    ],

    "feinty": [
        "leather", "tobacco", "plastic", "oily", "wax", "mineral", "earthy"
    ],

    "sulphury": [
        "menthol", "eucalyptus"
    ],

    "sweet": [
        "candy", "caramel", "chocolate", "cocoa", "cream", "custard",
        "honey", "sweet", "sugar", "syrup", "toffee", "rum"
    ],

    "nutty": [
        "almond", "hazelnut", "nut", "walnut", "marzipan"
    ],

    "wine": [
        "sherry", "wine", "raisin", "jam", "marmalad", "liqueur"
    ],

    "coastal": [
        "brine", "salt", "iodine"
    ],

    "roasted": [
        "coffee", "roast", "toast", "pastry", "bread", "cake"
    ]
    
}

df_tasting_notes_group = []

for tasting_notes_group in list(tasting_notes_groups.keys()):

    for tasting_note in tasting_notes_groups.get(tasting_notes_group):

        df_tasting_note_group = {
            'tasting_notes_group': tasting_notes_group,
            'tasting_note': tasting_note
        }

        df_tasting_notes_group.append(df_tasting_note_group)

df_tasting_notes_group = pd.DataFrame(df_tasting_notes_group)
df_tasting_notes_group

,tasting_notes_group,tasting_note
0,fruity,apple
1,fruity,apricot
2,fruity,banana
3,fruity,berry
4,fruity,blackberry
...,...,...
105,roasted,roast
106,roasted,toast
107,roasted,pastry
108,roasted,bread


In [207]:
df = df.merge(df_tasting_notes, how='left', on='review_title')
df = df.merge(df_tasting_notes_group, how='left', on='tasting_note')
df

,review_title,review_full_text,review_full_text_processed,review_text_vector,tasting_note,has_tasting_note,tasting_notes_group
0,"A Secret Highland Distillery 14 yo 2010 (54,1%...","a lot of sherry influence, but never in your ...",lot sherry influenc never face first tart goos...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",almond,0,nutty
1,"A Secret Highland Distillery 14 yo 2010 (54,1%...","a lot of sherry influence, but never in your ...",lot sherry influenc never face first tart goos...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",anise,0,spicy
2,"A Secret Highland Distillery 14 yo 2010 (54,1%...","a lot of sherry influence, but never in your ...",lot sherry influenc never face first tart goos...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",apple,1,fruity
3,"A Secret Highland Distillery 14 yo 2010 (54,1%...","a lot of sherry influence, but never in your ...",lot sherry influenc never face first tart goos...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",apricot,0,fruity
4,"A Secret Highland Distillery 14 yo 2010 (54,1%...","a lot of sherry influence, but never in your ...",lot sherry influenc never face first tart goos...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",ash,0,peaty
...,...,...,...,...,...,...,...
696978,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","a complete flower bouquet, mixed with honey a...",complet floral bouquet mix honey fruit tangeri...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",warm,0,spicy
696979,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","a complete flower bouquet, mixed with honey a...",complet floral bouquet mix honey fruit tangeri...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wax,0,feinty
696980,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","a complete flower bouquet, mixed with honey a...",complet floral bouquet mix honey fruit tangeri...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wine,0,wine
696981,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","a complete flower bouquet, mixed with honey a...",complet floral bouquet mix honey fruit tangeri...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wood,0,woody


In [211]:
freq = review_text_vec.sum(axis=0)

freq_df = pd.DataFrame({
    'word': counter.get_feature_names_out(),
    'frequency': freq
}).sort_values(by='frequency', ascending=False).reset_index(drop=True)

print(freq_df)

              word  frequency
0            fruit       4084
1            sweet       3619
2              oak       2927
3           pepper       2908
4            spice       2815
..             ...        ...
106        varnish        322
107     gooseberry        313
108       hazelnut        309
109         iodine        301
110  passion fruit          0

[111 rows x 2 columns]


In [217]:
words_freq = (review_text_vec.T @ review_text_vec)

df_words_freq = pd.DataFrame(words_freq, columns=tasting_notes_words_filtered, index=tasting_notes_words_filtered)
np.fill_diagonal(df_words_freq.values, -1)

df_words_freq

,almond,anise,apple,apricot,ash,banana,barley,berry,bitter,blackberry,...,toffee,tropic,vanilla,varnish,walnut,warm,wax,wine,wood,zest
almond,-1,64,435,182,97,218,75,94,240,32,...,184,121,536,43,162,133,246,58,324,136
anise,64,-1,208,73,38,122,45,66,108,23,...,65,58,232,34,75,57,92,30,202,77
apple,435,208,-1,345,135,500,271,263,520,80,...,431,237,1292,96,306,252,566,139,805,421
apricot,182,73,345,-1,31,227,38,140,190,30,...,151,206,510,55,146,133,282,65,399,107
ash,97,38,135,31,-1,29,24,42,86,19,...,71,46,187,5,79,103,65,20,126,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
warm,133,57,252,133,103,140,48,71,110,25,...,112,100,356,17,96,-1,185,38,228,83
wax,246,92,566,282,65,319,93,87,277,22,...,107,250,561,33,170,185,-1,36,343,255
wine,58,30,139,65,20,52,19,126,110,50,...,100,35,135,5,75,38,36,-1,153,41
wood,324,202,805,399,126,474,110,371,556,163,...,412,258,1012,192,426,228,343,153,-1,231
